In [1]:
import sys; sys.path.append('..'); sys.path.append('../..')
from common.postgresql import PostgresConnector
import pandas as pd
import re

db = PostgresConnector()
read_sql = db.read_sql
sql = db.execute_sql

# Leitura de Dados
Lendo gestores da CVM. A query foi construída para facilitar a inserção de novas tabelas via UNION.

In [ ]:
query = """
SELECT DISTINCT gestor, 'cvm.cadastro' as tabela 
FROM cvm.cadastro 
WHERE dt_fim IS NULL
"""
# Exemplo de como adicionar mais tabelas:
# query += """ UNION SELECT DISTINCT gestor, 'outra.tabela' as tabela FROM outra.tabela ... """

df_gestores = read_sql(query)
print(f"Total de gestores encontrados: {len(df_gestores)}")
df_gestores.head()

# Definição de Padrões (De-Para)
Lista de tuplas `(Grupo, Regex Pattern)`. 
**Regra para nomes curtos (< 4 chars):** São tratados como palavras inteiras (match exato ou cercado por espaços) para evitar falsos positivos (ex: 'BB' não deve pegar 'BBA').

In [ ]:
# Lista de padrões. A ordem importa (primeiro match vence).
patterns = [
    ('Itaú', r'Ita(ú|u)'),
    ('Intrag', r'Intrag'), # Ajustado para diferenciar do Itaú, ou se for para capturar 'Intrag' com regra específica
    ('BB', r'BB'),
    ('Bradesco', r'Bradesco'),
    ('Caixa', r'Caixa'),
    ('Safra', r'Safra'),
    ('Santander', r'Santander'),
    ('BTG', r'BTG'),
    ('XP', r'XP'),
    ('Vinci', r'Vinci'),
    ('Credit Suisse', r'Credit Suisse'),
    ('Credit Suisse', r'CSHG'),
    ('BofA', r'Bank of America'),
    ('Citibank', r'Citibank'),
    ('Citibank', r'Citi'),
    ('Daycoval', r'Daycoval'),
    ('Bocom', r'Bocom'),
    ('Opportunity', r'Opportunity'),
    ('Pátria', r'P(á|a)tria'),
    ('Kinea', r'Kinea'),
    ('ARX', r'ARX')
]

# Nota: O usuário pediu especificamente: [('Itaú', r'Ita(ú|u)'), ('Intrag', r'Ita(ú|u)')]
# Se o objetivo for mapear 'Ita(ú|u)' para Intrag também, isso geraria conflito se estiverem na mesma lista.
# Mantendo a lógica de pattern matching robusta e prioridade de lista.

In [ ]:
def aplicar_depara(nome, patterns_list):
    if not isinstance(nome, str):
        return None
        
    # Normalização básica para ajudar no match (opcional, dependendo do regex)
    # nome_upper = nome.upper()
    
    for grupo, pattern in patterns_list:
        # Tratamento para nomes curtos (< 4 chars)
        regex_pattern = pattern
        if len(pattern) < 4:
            # Garante que é uma palavra sozinha: Começo de string ou espaço + pattern + Final de string ou fim de palavra/espaço
            # Usando word boundaries \b é bom, mas para caracteres acentuados ou específicos, o controle manual de espaço é mais seguro conforme pedido.
            # Pedido: "ou sozinho ou com um espaço"
            regex_pattern = r'(?:^|\s)' + pattern + r'(?:$|\s)'
        
        # Busca case insensitive
        if re.search(regex_pattern, nome, re.IGNORECASE):
            return grupo
            
    return None # Ou retornar 'Outros' / o próprio nome se preferir

# Testes rápidos de sanidade
testes = [
    ("Itaú Unibanco", "Itaú"),
    ("Itaú BBA", "Itaú"),
    ("BB Gestão", "BB"),
    ("Prev BB", "BB"),
    ("UBBA", None), # Não deve pegar BB
    ("BBA", None), # Não deve pegar BB (patterns são cases sensiveis? regex no codigo acima é IGNORECASE)
]

print("Validando lógica em casos de teste:")
for entrada, esperado in testes:
    resultado = aplicar_depara(entrada, patterns)
    match_ok = (resultado == esperado) or (esperado is None and resultado is None)
    print(f"'{entrada}' -> {resultado} (Esperado: {esperado}) [{'OK' if match_ok else 'FALHA'}]")

# Aplicação no Dataset

In [ ]:
df_gestores['grupo'] = df_gestores['gestor'].apply(lambda x: aplicar_depara(x, patterns))

# Filtrar ou organizar resultado final
# O usuário pediu colunas: "grupo", "gestor", "tabela"
df_final = df_gestores[['grupo', 'gestor', 'tabela']].copy()

# Mostrar exemplos classificados
print("\nExemplos classificados:")
display(df_final[df_final['grupo'].notnull()].head(10))

# Mostrar exemplos NÃO classificados (para aprimorar os patterns)
print("\nExemplos NÃO classificados:")
display(df_final[df_final['grupo'].isnull()].head(10))